In [1]:
import random
import string
import faker
import math
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
random.seed(348)

This file aims to generate the dataset in use of the parking lots management database. There are 5 tables to be generated:

- User
- Permit_Holder
- Lot
- Spot
- Booking

## User

- <u>`uid`</u> (NOT NULL): uniquely identifies a user; no modifications from users allowed; no default value
- `username` (NOT NULL): randomly generated username; allows users to modify by themselves; no default value
- `password` (NOT NULL): randomly generated password; allows users to modify by themselves; no default value
- `identity` (NOT NULL): any of {"student", "staff", "visitor", "admin"}; no modifications from users allowed; max 5 admin users; no default value
- `email` (NOT NULL): randomly generated email; allows users to modify by themselves; no default value
- `phone`: randomly generated 10-digit phone number; allows users to modify by themselves; default value: `NULL`
- `booking_num` (NOT NULL): number of current bookings; no modifications from users allowed; default value: 0

### Helper functions

In [2]:
# To generate a list of random strings
def generate_rand_str_lst(iterable, weights=-1, num = 10, min_length = 8, max_length = 16):
    lst = []
    # Determine weights
    if weights == -1:
        p_count = len(iterable)
        weights = [1 / p_count for _ in range(p_count)] # equal probability of being selected
    else: # if user-specified
        weights = weights
        
    for _ in range(num):
        length = random.randint(min_length, max_length)
        p = ''.join(random.choices(population = iterable, weights = weights, k = length)) # To generate a random string of specific length and characters
        lst.append(p)
    return lst

### Main functions

In [3]:
# To generate a list of uid
def generate_uid(num=10, iniId=1):
    return [i + iniId for i in range(num)]

# To generate a list of username
def generate_username_lst(num = 10, 
                          iterable = string.ascii_letters + string.digits, 
                          weights = -1,
                          min_length = 8, max_length = 12):
    return generate_rand_str_lst(iterable, weights, num, min_length, max_length)

def generate_unique_usernames(num):
    fake = faker.Faker()
    usernames = set()

    while len(usernames) < num:
        username = fake.user_name()
        usernames.add(username)

    return list(usernames)

# To generate a list of password
def generate_password_lst(num = 10, 
                          iterable = string.ascii_letters + string.digits + "~`@#$%^&*_-+=:;'.", 
                          weights = -1,
                          min_length = 8, max_length = 16):
    return generate_rand_str_lst(iterable, weights, num, min_length, max_length)

# To generate a list of identity 
def generate_identity_lst(num = 10, types = ["student", "staff", "visitor"], weights = [0.4, 0.3, 0.3]):
    admin_num = random.randint(1, 5) # at least 1 admin, at most 5 admins regardless of size of dataset
    res = ["admin" for _ in range(admin_num)]
    return res + random.choices(population = types, weights = weights, k = num - admin_num)

# To generate a list of email
def genrate_email_lst(num = 10, iterable = string.ascii_letters + string.digits, weights = -1,
                      domains = ["gmail.com", "uwaterloo.ca", "yahoo.com", "hotmail.com", "outlook.com"],
                      min_length = 8, max_length = 12):
    prefix = generate_rand_str_lst(iterable, weights, num, min_length, max_length)
    suffix = [random.choice(domains) for _ in range(num)]
    return [f"{prefix[i]}@{suffix[i]}" for i in range(num)]

# To generate a list of random 10-digit phone number as a string
def generate_phone_number_lst(num = 10, null_probability = 0.2):
    lst = []
    for _ in range(num):
        if random.random() <= null_probability: # random.random() generates a floating number 0 <= x <= 1.0
            lst.append(None)  # append None to represent NULL in the database
        else:
            lst.append(str(random.randint(1000000000, 9999999999)))  # generate a random 10-digit phone number as a string
    return lst

# To generate a list of booking_num
def generate_booking_num_lst(num = 10, weights = [0.6, 0.2, 0.1, 0.1]):
    return random.choices(population = [0, 1, 2, 3], weights = weights, k = num)

# To generate data for user table
def generate_users(num = 10):
    fake = faker.Faker()
    uids = generate_uid(num)
    #usernames = generate_username_lst(num)
    usernames = generate_unique_usernames(num)
    passwords = generate_password_lst(num)
    identities = generate_identity_lst(num)
    emails = genrate_email_lst(num)
    phonenums = generate_phone_number_lst(num)
    booking_nums = generate_booking_num_lst(num)
    accessible = random.choices(population = [True, False], weights = [0.1, 0.9], k = num)
    return [(uids[i], usernames[i], passwords[i], emails[i], phonenums[i], identities[i], booking_nums[i], accessible[i]) for i in range(num)]

# Lots (CW)

- lid
- area,
- address,
- lot_name,
- capacity,
- like_num

In [4]:
# First few rows are 2h-free

# forth last row paid $1.5/h - max 24h
# third last $2/h - max 3h
# second last row paid $2.5/h - max 24h
# last row paid $3/h - max 24h
adds = [
    '550 King St N','555 Davenport Rd', '600 King St N',
    '45 Erb St W', '100 Regina St S', '70 Caroline St S', 
    '31 Caroline St N', '20 Regina St S', '82 Erb St',
    '165 Weber St S', '170 Weber St S', '70 Bridgeport Rd E', 
    '550 King St N', '550 King St N', '550 King St N',
    'Bloomingdale', '2001 University Ave E',
    '410 The Boardwalk', 
    '35 Albert St', 'Woolwich St N',
    '10 Wills Way'
]
lnames = [
    'Galaxy Cinemas Waterloo','Zehrs Conestoga', 'The Home Depot',
    'Waterloo Town Square (North)', 'City Centre Lot', 'Waterloo Town Square (South)',
    'Perimeter Lot', 'Station Lot', 'Museum Lot',
    'Forbes Motors Chevrolet Buick GMC', 'Carimex Auto Sales', 'Walmart Supercentre', 
    'Conestoga Mall I', 'Conestoga Mall II', 'Conestoga Mall III', 
    'Elam Martin Farmstead at RIM Park', 'RIM Park',
    'Milestones', 
    'Library Lot', 'Grey Silo Golf Club',
    'Uptown Parkade'
]
capas = [
    250,250, 200,
    40, 115, 200, 
    50, 50, 100,
    150, 150, 300,
    50, 250, 250,
    50, 250, 
    150,
    150, 55,
    50
]
like_num = [
    1037, 362, 1032,
    0, 302, 407,
    502, 103, 301,
    1020, 400, 123, 
    87, 547, 994,
    11, 1200,
    303,
    1310, 80,
    289
]
lids = [i + 30 for i in range(len(lnames))]
areas = ['CW' for i in range(len(lnames))]

In [5]:
def generate_lots(lids, areas, adds, lnames, capas, like_num):
    res = []
    for i in range(len(lids)):
        res.append((lids[i], areas[i], adds[i], lnames[i], capas[i], like_num[i]))
    return res

# Spots (CW)

- lid
- sid
- parking_type
- latitude
- longitude
- max_stay
- price

In [6]:
# 4 points to highlight the lot on map
lat_long = [
    [(43.495766, -80.527780), (43.496095, -80.524909), (43.495160, -80.524826), (43.494998, -80.527072)],
    [(43.498200, -80.524452), (43.497621, -80.525088), (43.496675, -80.524028), (43.497254, -80.523446)],
    [(43.504435, -80.534556), (43.503771, -80.534935), (43.502976, -80.534281), (43.503401, -80.533365)],
    [(43.464236, -80.523939), (43.464482, -80.523869), (43.464553, -80.523183), (43.464303, -80.523130)], 
    [(43.463883, -80.519793), (43.463668, -80.519393), (43.464509, -80.519100), (43.464380, -80.518615)],
    [(43.461641, -80.522792), (43.461895, -80.522183), (43.462370, -80.523378), (43.462702, -80.522582)],
    [(43.465385, -80.527146), (43.464997, -80.527137), (43.465387, -80.526506), (43.465015, -80.526736)],
    [(43.465111, -80.520909), (43.464752, -80.520848), (43.464818, -80.520272), (43.465171, -80.520375)],
    [(43.464394, -80.527087), (43.464319, -80.527242), (43.464034, -80.526859), (43.464177, -80.526343)],
    [(43.463720, -80.504123), (43.462751, -80.503482), (43.463236, -80.502071), (43.463987, -80.503357)],
    [(43.463751, -80.502495), (43.463373, -80.501864), (43.463529, -80.501426), (43.463967, -80.501893)],
    [(43.470704, -80.514375), (43.469473, -80.516179), (43.469060, -80.515877), (43.470059, -80.513522)],
    [(43.498743, -80.526403), (43.498259, -80.525038), (43.499426, -80.526362), (43.498553, -80.524529)],
    [(43.500013, -80.528558), (43.500001, -80.528118), (43.498195, -80.529169), (43.498191, -80.528322)],
    [(43.497366, -80.529019), (43.497347, -80.528327), (43.496078, -80.528810), (43.496024, -80.528322)],
    [(43.517386, -80.494412), (43.517262, -80.494452), (43.516987, -80.493564), (43.517105, -80.493419)],
    [(43.518255, -80.502704), (43.517849, -80.502452), (43.518092, -80.500494), (43.519164, -80.500566)],
    [(43.439943, -80.565112), (43.439164, -80.564319), (43.439679, -80.563367), (43.440045, -80.563762)],
    [(43.465812, -80.525633), (43.465384, -80.525451), (43.465544, -80.524928), (43.465953, -80.525266)],
    [(43.517632, -80.493981), (43.517488, -80.494057), (43.516971, -80.492691), (43.517168, -80.492510)],
    [(43.463665, -80.521261), (43.463293, -80.521191), (43.463339, -80.520806), (43.463719, -80.520892)],
]

In [7]:
# To export coordinates for highlighting lots in CW
#df = pd.DataFrame(lat_long)
#df.to_csv('CW_lot_coords.txt', index=False, header=False)

In [8]:
# find ranges to generate long and lat for each spot
def find_pair_range(lst):
    latitudes = [lat for lat, lon in lst]
    longitudes = [lon for lat, lon in lst]
    
    min_lat = min(latitudes)
    max_lat = max(latitudes)
    min_long = min(longitudes)
    max_long = max(longitudes)
    
    return {
        "min_lat": min_lat,
        "max_lat": max_lat,
        "min_long": min_long,
        "max_long": max_long
    }

def find_all_ranges(lat_long_lst):
    results = []
    for lst in lat_long_lst:
        results.append(find_pair_range(lst))
    return results

In [9]:
lid_s = []
sid = []
parking_type = []
lat = []
long = []
max_stay = []
price = []

In [10]:
# generate lid and sid in spots table
for i in range(len(lnames)):
    for j in range(capas[i]):
        lid_s.append(lids[i])
        sid.append(j + 1)

In [11]:
# generate lat and long for each spot
all_ranges = find_all_ranges(lat_long)

for i,r in enumerate(all_ranges):
    for j in range(capas[i]):
        lat.append(random.uniform(r['min_lat'], r['max_lat']))
        long.append(random.uniform(r['min_long'], r['max_long']))

In [12]:
# generate type and price for spots
for i,c in enumerate(capas):
    ac = math.floor(c * 0.1)
    if i < 10:
        weights = [1, 0]
        prices = [0, 0]
        maxs = [2, 2]
    elif 10 <= i and i < 16:
        weights = [0.3, 0.7]
        prices = [0, 1.5]
        maxs = [2, random.choices(population = [1.5, 2, 3, 4], weights = [0.1, 0.3, 0.5, 0.1], k = 1)[0]]
    elif 16 <= i and i < 18:
        weights = [0.4, 0.6]
        prices = [0, random.choices(population = [1.5, 2, 3], weights = [0.4, 0.3, 0.3], k = 1)[0]]
        maxs = [2, random.choices(population = [1.5, 2, 3, 4], weights = [0.4, 0.3, 0.2, 0.1], k = 1)[0]]
    elif 18 <= i and i < 20:
        weights = [0.5, 0.5]
        prices = [0, random.choices(population = [1.25, 1.5, 3], weights = [0.4, 0.3, 0.3], k = 1)[0]]
        maxs = [2, random.choices(population = [1.5, 2, 3, 4], weights = [0.4, 0.3, 0.2, 0.1], k = 1)[0]]
    else:
        weights = [0, 1]
        prices = [0, random.choices(population = [2, 2.5, 3], weights = [0.4, 0.3, 0.3], k = 1)[0]]
        maxs = [2, random.choices(population = [1.5, 2, 3, 4], weights = [0.4, 0.3, 0.2, 0.1], k = 1)[0]]
    acv = random.choices(population = [2, 3, 4], weights = [0.4, 0.3, 0.3], k = 1)[0]
    for _ in range(ac):
        parking_type.append('accessible')
        price.append(0)
        max_stay.append(acv)
    for _ in range(c - ac):
        v = random.choices(population = ['free', 'pay'], weights = weights, k = 1)[0]
        if v == 'free':
            p = prices[0]
            ms = maxs[0]
        else:
            p = prices[1]
            ms = maxs[1]
        parking_type.append(v)
        price.append(p)
        max_stay.append(ms)

In [13]:
def generate_spots(lid_s, sid, parking_type, lat, long, max_stay, price):
    res = []
    for i in range(len(lid_s)):
        res.append((lid_s[i], sid[i], parking_type[i], lat[i], long[i], max_stay[i], price[i]))
    return res

# Export

In [14]:
users = generate_users(1000)
lots = generate_lots(lids, areas, adds, lnames, capas, like_num)
spots = generate_spots(lid_s, sid, parking_type, lat, long, max_stay, price)

In [15]:
headers_u = ['uid', 'username', 'password', 'email', 'phone', 'identity', 'booking_num', 'is_accessible']
headers_l = ['lid', 'area', 'address', 'lot_name', 'capacity', 'like_num']
header_s = ['lid', 'sid', 'parking_type', 'latitude', 'longitude', 'max_stay', 'price']

In [16]:
user_df = pd.DataFrame(users, columns = headers_u)
lot_df = pd.DataFrame(lots, columns = headers_l)
spot_df = pd.DataFrame(spots, columns = header_s)

In [17]:
# Function to generate expiry date
def generate_expiry_date(expired = False):
    baseline_date = datetime(2024, 7, 25)
    if expired:
        return baseline_date - timedelta(days=random.randint(1, 365))
    else:
        return baseline_date + timedelta(days=random.randint(1, 365))

In [18]:
# Filter the users with identity as 'student' or 'staff'
filtered_df = user_df.loc[user_df['identity'].isin(['student', 'staff'])].copy()
filtered_df['type'] = filtered_df['identity']
# Generate expiry dates with 80% not expired and 20% expired
filtered_df['expiry_date'] = filtered_df.apply(lambda x: generate_expiry_date(expired=random.random() < 0.2), axis=1)
filtered_df['expiry_date'] = filtered_df['expiry_date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))

permit_holders_df = filtered_df[['uid', 'type', 'expiry_date']]
permit_holders_df.columns = ['uid', 'type', 'expiry_date']

In [19]:
#user_df.to_csv('user.txt', index=False)
#permit_holders_df.to_csv('p_holders.txt', index=False)
#lot_df.to_csv('lots_CW.txt', index=False)
#spot_df.to_csv('spots_CW.txt', index=False)